# Manually Regridding OSOM Data

The OSOM data (model output and bathymetry) uses it's own internal grid for modelling. To have a usable output, the data needs to be transformed into a more "normal" grid that can be georeferenced and tiled.

In [1]:
# Import Libraries used in Regridding
import netCDF4 as nc
import numpy as np

# For Display
import datetime

# Image Creation (demo)
from PIL import Image
from IPython.display import display
from matplotlib.pyplot import imshow

File paths for transformation

In [2]:
# Define useful parameters
grid_path = "/oscar/data/epscor/OSOM/input/ROMS_forcing_files/grid/osom_grid4_mindep_smlp_mod10.nc"
data_path = "/oscar/data/epscor/OSOM/output/OSOM_v2/2022/"
data_filename = "ocean_his_6210.nc"
data_varname = (
    "temp"  # variable names include temp, salt, zeta, ubar_eastward, ubar_westward
)

## Import OSOM Grid & Dataset

In [3]:
# Import OSOM Grid Data
osom_grid = nc.Dataset(grid_path)

lon = osom_grid.variables["lon_rho"][:]
lat = osom_grid.variables["lat_rho"][:]
mask = osom_grid.variables["mask_rho"][:]
bathy = osom_grid.variables["h"][:]

In [4]:
# Import Data File
ds = nc.Dataset(data_path + data_filename)
#
time = ds.variables["ocean_time"][:]
data = ds.variables[data_varname][
    :, -1, :, :
]  # for surface layers: [:,-1,:,:]; for bottom layers: [:,0,:,:]


# Function to translate OSOM date (calculated as consecutive days from 1 Jan 2005)
# and convert it to a generic date in time in a format that is useful
# for plotting.
def get_date(idx):
    date = datetime.datetime(2005, 1, 1) + datetime.timedelta(seconds=time[idx])
    date_string = date.strftime(
        "%d %b %Y %H:%M"
    )  # https://strftime.org/ for a cheat sheet
    return date_string


timepoint = 1
time_string = get_date(timepoint)
data_at_timepoint = data[timepoint]

The model data that OSOM outputs eixsts on an almost trapezoidial grid. That means that each OSOM grid cell is a lot smaller (in a lat/lon) sense the more north you go. Given this, here's the basic algorith to perform this transformation.

1. Find the lat / lon bounds of the dataset.
2. Determine the pixel count for the output image. (OSOM data comes in 1000x1100, but for this transformed data, there's going to be a lot more dead space, so the pixel count should probably be larger.)
3. Create a 2D array of the size determined in step 2.
4. For each element in this array:
   1. Determine the lat/lon coordinate point of this element.
   2. Use the grid to determine the xi/eta coordinates of that point (the internal grid system used by the OSOM dataset).
   3. If a data point is found, place the model value at current element of the output image.
   4. Otherwise, place a null value at that element.

Because of the shifting scale of the OSOM data points, the process of "selecting the correct model value" will probably require averaging across several OSOM data point.

In [5]:
# Compute Lat / Lon bounds

lat_min = np.min(lat)
lat_max = np.max(lat)
lon_min = np.min(lon)
lon_max = np.max(lon)

lat_range = lat_max - lat_min
lon_range = lon_max - lon_min

print(
    f"Bounds: {round(lat_min, 2)} to {round(lat_max, 2)} (lat) {round(lon_min, 2)} to {round(lon_max, 2)} (lon)"
)

Bounds: 40.51 to 42.17 (lat) -72.67 to -69.99 (lon)


In [40]:
# Establish output raster size

print(lat_range, lon_range)

output_dim_x = 26 * 100  # Small values for test
output_dim_y = 16 * 100

1.6670863969781493 2.680264108770743


In [41]:
# Get model data for output point

lat_cell_size = lat_range / output_dim_y
lon_cell_size = lon_range / output_dim_x


def get_coordinates_for_point(x, y):
    cell_lon_min = (x * lon_cell_size) + lon_min
    cell_lon_max = ((x + 1) * lon_cell_size) + lon_min
    cell_lat_min = (y * lat_cell_size) + lat_min
    cell_lat_max = ((y + 1) * lat_cell_size) + lat_min
    return (cell_lon_min, cell_lon_max, cell_lat_min, cell_lat_max)


def get_data_for_points(x_mask, y_mask, dataset):
    # Note (AM): This is a method that's a little flawed right now. This will collect
    # all points enclosed by a lat/lon region. For low density regriddings, it's
    # totally okay, but for higher density ones it's a little worse. For example, it's
    # possible to imagine a dataset pixel that encloses entirely a regridding pixel, which
    # would be skipped under the current algorithm. In effect, this only detects the *edges*
    # of dataset pixels, not the pixels themselves.
    collected_data = []
    for i in range(len(x_mask)):
        data = dataset[x_mask[i]][y_mask[i]]
        collected_data.append(data)
    return collected_data


def get_model_data_at_point(x, y, grid_lon, grid_lat, dataset):
    cell_lon_min, cell_lon_max, cell_lat_min, cell_lat_max = get_coordinates_for_point(
        x, y
    )
    lon_mask = (grid_lon > cell_lon_min) & (grid_lon < cell_lon_max)
    lat_mask = (grid_lat > cell_lat_min) & (grid_lat < cell_lat_max)
    x_mask, y_mask = np.where(lon_mask & lat_mask)
    bounded_points = get_data_for_points(x_mask, y_mask, dataset)
    if len(bounded_points) >= 1:
        averaged_points = np.average(bounded_points)
        return averaged_points
    else:
        return np.nan

In [ ]:
def populate_regrid(size_x, size_y, grid_lon, grid_lat, dataset):
    regrid = np.empty((size_x, size_y))
    regrid.fill(np.nan)
    for x in range(size_x):
        print(x, "/", size_x)
        for y in range(size_y):
            regrid[x][y] = get_model_data_at_point(x, y, grid_lon, grid_lat, dataset)
    return regrid


regridded = populate_regrid(output_dim_x, output_dim_y, lon, lat, data_at_timepoint)
# print(regridded)

0 / 2600
1 / 2600
2 / 2600
3 / 2600
4 / 2600
5 / 2600


In [ ]:
values = np.unique(regridded[~np.isnan(regridded)])
output_min = np.min(values)
output_max = np.max(values)


def normalize(v, v_scale_min, v_scale_max, o_scale_min, o_scale_max):
    standard_normalization = (v - v_scale_min) / (v_scale_max - v_scale_min)
    return ((o_scale_max - o_scale_min) * standard_normalization) + o_scale_min


def create_image(dataset):
    # Create a base transparent image
    image = Image.new(
        mode="RGBA", size=(output_dim_x, output_dim_y), color=(0, 0, 0, 0)
    )
    for x in range(output_dim_x):
        for y in range(output_dim_y):
            value = dataset[x][y]
            if not np.isnan(value):
                color_as_value = int(normalize(value, output_min, output_max, 0, 255))
                image.putpixel(
                    (x, output_dim_y - 1 - y), (color_as_value, 0, color_as_value, 255)
                )
    return image


im = create_image(regridded)
display(im)
imshow(np.asarray(im))
im.save(f"{output_dim_x}x{output_dim_y}-databay.png")